# Results Aggregation (For Classification Accuracy and Loss)

This notebook is for aggregating results of classification accuracy and loss under same experiment on multiple global seeds. 

Please set all the variables following the instructions and Hit "Run All". 

## 1. Specify Result Folders

*We need you to group the folders in a seperate folder in advance. We will generate aggregated results under this folder.*

Please specify this group folder path.

In [63]:
group_folder_path: str = "../outputs_critical/Zhu_2025_03_05_seed_1-10_til_pmnist_cbphat_20_tasks/"

(Optional) If you want to aggregate all the results in the folder, please set `if_all` as `True`. Otherwise, set it `False` and specify these result folder names.

In [64]:
if_all: bool = True
result_folder_names: list[str] = []

Please specify the relative path (with respect to result folder) of both accuracy and loss CSV files. Default is `results/acc.csv` and `results/loss_cls.csv`.

In [65]:
acc_relative_path: str = "results/acc.csv"
loss_cls_relative_path: str = "results/loss_cls.csv"

Building paths...

In [66]:
import os

if if_all:
    result_folder_names = [f for f in os.listdir(group_folder_path) if os.path.isdir(os.path.join(group_folder_path, f)) and f != "aggregated_results"]
    
acc_paths = [os.path.join(group_folder_path, f, acc_relative_path) for f in result_folder_names]
loss_cls_paths = [os.path.join(group_folder_path, f, loss_cls_relative_path) for f in result_folder_names]

## 2. Read and Process the CSV

We read the CSV files into Pandas Dataframes and calculate the average and standard deviation.

In [67]:
import pandas as pd

# dataframes to store results
acc_df = []
loss_cls_df = []

# Read and aggregate all CSV files
for acc_file, loss_cls_file in zip(acc_paths, loss_cls_paths):
    acc_df.append(pd.read_csv(acc_file))
    loss_cls_df.append(pd.read_csv(loss_cls_file))
    
acc_df_concat = pd.concat(acc_df, ignore_index=True)
loss_cls_df_concat = pd.concat(loss_cls_df, ignore_index=True)

acc_mean_df = acc_df_concat.groupby("after_training_task").mean()
acc_std_df = acc_df_concat.groupby("after_training_task").std()

loss_cls_mean_df = loss_cls_df_concat.groupby("after_training_task").mean()
loss_cls_std_df = loss_cls_df_concat.groupby("after_training_task").std()
    

## 3. Save and Plot

Please specify the relative path (with respect to group folder) of both aggregated accuracy and loss CSV files and plots.

In [68]:
# CSV files relative paths
acc_mean_relative_path = "aggregated_results/acc_mean.csv"
loss_cls_mean_relative_path = "aggregated_results/loss_cls_mean.csv"
acc_std_relative_path = "aggregated_results/acc_std.csv"
loss_cls_std_relative_path = "aggregated_results/loss_cls_std.csv"

# Plots relative paths
acc_matrix_mean_relative_path = "aggregated_results/acc_matrix_mean.png"
loss_cls_matrix_mean_relative_path = "aggregated_results/loss_cls_matrix_mean.png"
ave_acc_mean_plot_relative_path = "aggregated_results/ave_acc_mean.png"
ave_loss_cls_mean_plot_relative_path = "aggregated_results/ave_loss_cls_mean.png"

Building paths...

In [69]:
if not os.path.exists(os.path.join(group_folder_path, "aggregated_results")):
    os.makedirs(os.path.join(group_folder_path, "aggregated_results"))

acc_mean_path = os.path.join(group_folder_path, acc_mean_relative_path)
loss_cls_mean_path = os.path.join(group_folder_path, loss_cls_mean_relative_path)
acc_std_path = os.path.join(group_folder_path, acc_std_relative_path)
loss_cls_std_path = os.path.join(group_folder_path, loss_cls_std_relative_path)

acc_matrix_mean_path = os.path.join(group_folder_path, acc_matrix_mean_relative_path)
loss_cls_matrix_mean_path = os.path.join(group_folder_path, loss_cls_matrix_mean_relative_path)
ave_acc_mean_plot_path = os.path.join(group_folder_path, ave_acc_mean_plot_relative_path)
ave_loss_cls_mean_plot_path = os.path.join(group_folder_path, ave_loss_cls_mean_plot_relative_path)


Save the aggregated results as CSV files.

In [70]:
acc_mean_df.to_csv(acc_mean_path)
acc_std_df.to_csv(acc_std_path)
loss_cls_mean_df.to_csv(loss_cls_mean_path)
loss_cls_std_df.to_csv(loss_cls_std_path)

Plot the aggregated results and save the figures.

In [71]:
from clarena import utils

utils.plot_test_acc_matrix_from_csv(csv_path=acc_mean_path, plot_path=acc_matrix_mean_path)
utils.plot_test_loss_cls_matrix_from_csv(csv_path=loss_cls_mean_path, plot_path=loss_cls_matrix_mean_path)
utils.plot_test_ave_acc_curve_from_csv(csv_path=acc_mean_path, plot_path=ave_acc_mean_plot_path)
utils.plot_test_ave_loss_cls_curve_from_csv(csv_path=loss_cls_mean_path, plot_path=ave_loss_cls_mean_plot_path)